
## Hopfield Networks

* can be use for anomaly detection


In [1]:

import torch
import numpy as np



In [2]:

class HopfieldNet:
    
    def __init__(self, size):
        self.size    = size
        self.weights = np.zeros( (size, size) )
    
    def train(self, patterns):
        
        for p in patterns:
            ## self.weights += np.outer(p, p)
            self.weights = self.weights + np.outer(p, p)
            
        np.fill_diagonal( self.weights, 0 )     # No self-connections
        ## self.weights /= len(patterns)
        self.weights = self.weights / len(patterns)
    
    def reconstruct(self, pattern, steps=10):
        
        for _ in range(steps):
            pattern = np.sign(   np.dot(  self.weights, pattern)   )
            
        return pattern


In [3]:


data = np.array(
    [
    [1, 1, 1, -1, 1, 1],  # Sample 1 (normal)
    [1, 1, -1, 1, 1, 1],  # Sample 2 (normal)
    [1, 1, 1, -1, 1, 1],  # Sample 3 (normal)
    [1, 1, 1, -1, 1, 1],  # Sample 4 (normal)
    ]
)


In [4]:

print( data.shape )

data


(4, 6)


array([[ 1,  1,  1, -1,  1,  1],
       [ 1,  1, -1,  1,  1,  1],
       [ 1,  1,  1, -1,  1,  1],
       [ 1,  1,  1, -1,  1,  1]])

In [5]:

hopfield = HopfieldNet( size=6 )

hopfield.weights


array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [6]:

hopfield.train( data )



## Test Samples


In [7]:

def hamming_distance(a, b):
    """
    Calculate the Hamming distance between two strings.
    """

    ## arr1 = np.array(list(str1), dtype=int)
    
    # XOR to find differing bits, then count the number of 1s
    return np.count_nonzero(a != b)




## Normal test


In [8]:

# Test 

test_sample     = [1,  1, 1, -1, 1, 1]
recalled_sample = hopfield.reconstruct(test_sample)

print("Test sample:     ",         test_sample)
print("Recalled sample: ", recalled_sample)

test_sample_tr     = torch.from_numpy( np.array( test_sample  )   )
recalled_sample_tr = torch.from_numpy( recalled_sample )


distance = hamming_distance(test_sample_tr, recalled_sample_tr)
distance


Test sample:      [1, 1, 1, -1, 1, 1]
Recalled sample:  [ 1.  1.  1. -1.  1.  1.]


0


## Abnormal test


In [9]:

# Test 

test_sample     = [-1, -1, 1, -1, -1, -1]
recalled_sample = hopfield.reconstruct(test_sample)

print("Test sample:     ",         test_sample)
print("Recalled sample: ", recalled_sample)

test_sample_tr     = torch.from_numpy( np.array( test_sample  )   )
recalled_sample_tr = torch.from_numpy( recalled_sample )

distance = hamming_distance(test_sample_tr, recalled_sample_tr)
distance



Test sample:      [-1, -1, 1, -1, -1, -1]
Recalled sample:  [-1. -1. -1.  1. -1. -1.]


2


## Breaking down the operations 


In [10]:

size = 6
weights = np.zeros( (size, size) )

for p in data:
    print( p )
    print( np.outer(p, p) )
    weights = weights + np.outer(p, p)
            
    

[ 1  1  1 -1  1  1]
[[ 1  1  1 -1  1  1]
 [ 1  1  1 -1  1  1]
 [ 1  1  1 -1  1  1]
 [-1 -1 -1  1 -1 -1]
 [ 1  1  1 -1  1  1]
 [ 1  1  1 -1  1  1]]
[ 1  1 -1  1  1  1]
[[ 1  1 -1  1  1  1]
 [ 1  1 -1  1  1  1]
 [-1 -1  1 -1 -1 -1]
 [ 1  1 -1  1  1  1]
 [ 1  1 -1  1  1  1]
 [ 1  1 -1  1  1  1]]
[ 1  1  1 -1  1  1]
[[ 1  1  1 -1  1  1]
 [ 1  1  1 -1  1  1]
 [ 1  1  1 -1  1  1]
 [-1 -1 -1  1 -1 -1]
 [ 1  1  1 -1  1  1]
 [ 1  1  1 -1  1  1]]
[ 1  1  1 -1  1  1]
[[ 1  1  1 -1  1  1]
 [ 1  1  1 -1  1  1]
 [ 1  1  1 -1  1  1]
 [-1 -1 -1  1 -1 -1]
 [ 1  1  1 -1  1  1]
 [ 1  1  1 -1  1  1]]



## np.outer

* np.outer produces one product between each possible element pairing from 2 tensors (a and b).
* np.outer is used for the cross product 
* the intuition is like a pairwise distance matrix
* this can help us to find the distance between each pair of point values
* Hebbian learning rule for training 


In [11]:

np.fill_diagonal( weights, 0 )     

weights
    


array([[ 0.,  4.,  2., -2.,  4.,  4.],
       [ 4.,  0.,  2., -2.,  4.,  4.],
       [ 2.,  2.,  0., -4.,  2.,  2.],
       [-2., -2., -4.,  0., -2., -2.],
       [ 4.,  4.,  2., -2.,  0.,  4.],
       [ 4.,  4.,  2., -2.,  4.,  0.]])

In [12]:

len(data)


4

In [13]:

## self.weights /= len(patterns)
weights = weights / len(data)


In [14]:

weights



array([[ 0. ,  1. ,  0.5, -0.5,  1. ,  1. ],
       [ 1. ,  0. ,  0.5, -0.5,  1. ,  1. ],
       [ 0.5,  0.5,  0. , -1. ,  0.5,  0.5],
       [-0.5, -0.5, -1. ,  0. , -0.5, -0.5],
       [ 1. ,  1. ,  0.5, -0.5,  0. ,  1. ],
       [ 1. ,  1. ,  0.5, -0.5,  1. ,  0. ]])


## Example assumming samples of images size (10, 10)


In [15]:

my_array = [0] * 50 + [1] * 50

my_array = np.array( my_array )

my_array


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [16]:

np.random.shuffle( my_array )

image = my_array

image = image.reshape((10,10))

print( image.shape )

image


(10, 10)


array([[1, 1, 1, 0, 1, 1, 0, 0, 1, 0],
       [1, 1, 0, 1, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
       [1, 1, 0, 1, 0, 0, 1, 1, 1, 0],
       [1, 0, 1, 0, 0, 1, 0, 1, 0, 0],
       [0, 1, 0, 1, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 1, 1, 0, 1, 1, 1],
       [1, 0, 0, 0, 0, 1, 1, 1, 1, 0],
       [1, 0, 1, 0, 0, 1, 0, 1, 0, 0],
       [1, 0, 0, 1, 0, 1, 1, 1, 0, 1]])

In [17]:

states = image * 2 - 1     ##  to polar values of (-1, 1)


In [18]:

states


array([[ 1,  1,  1, -1,  1,  1, -1, -1,  1, -1],
       [ 1,  1, -1,  1,  1, -1,  1,  1, -1, -1],
       [-1, -1, -1, -1,  1,  1,  1, -1, -1, -1],
       [ 1,  1, -1,  1, -1, -1,  1,  1,  1, -1],
       [ 1, -1,  1, -1, -1,  1, -1,  1, -1, -1],
       [-1,  1, -1,  1,  1, -1, -1, -1, -1,  1],
       [-1, -1, -1,  1,  1,  1, -1,  1,  1,  1],
       [ 1, -1, -1, -1, -1,  1,  1,  1,  1, -1],
       [ 1, -1,  1, -1, -1,  1, -1,  1, -1, -1],
       [ 1, -1, -1,  1, -1,  1,  1,  1, -1,  1]])

In [19]:

states = states.flatten() 

states


array([ 1,  1,  1, -1,  1,  1, -1, -1,  1, -1,  1,  1, -1,  1,  1, -1,  1,
        1, -1, -1, -1, -1, -1, -1,  1,  1,  1, -1, -1, -1,  1,  1, -1,  1,
       -1, -1,  1,  1,  1, -1,  1, -1,  1, -1, -1,  1, -1,  1, -1, -1, -1,
        1, -1,  1,  1, -1, -1, -1, -1,  1, -1, -1, -1,  1,  1,  1, -1,  1,
        1,  1,  1, -1, -1, -1, -1,  1,  1,  1,  1, -1,  1, -1,  1, -1, -1,
        1, -1,  1, -1, -1,  1, -1, -1,  1, -1,  1,  1,  1, -1,  1])

In [20]:

## no.outer does the cross product

weights = np.outer(states, states.T)
print( weights.shape )
weights 


(100, 100)


array([[ 1,  1,  1, ...,  1, -1,  1],
       [ 1,  1,  1, ...,  1, -1,  1],
       [ 1,  1,  1, ...,  1, -1,  1],
       ...,
       [ 1,  1,  1, ...,  1, -1,  1],
       [-1, -1, -1, ..., -1,  1, -1],
       [ 1,  1,  1, ...,  1, -1,  1]])

In [21]:

np.fill_diagonal(weights, 0)
print( weights.shape )
weights 


(100, 100)


array([[ 0,  1,  1, ...,  1, -1,  1],
       [ 1,  0,  1, ...,  1, -1,  1],
       [ 1,  1,  0, ...,  1, -1,  1],
       ...,
       [ 1,  1,  1, ...,  0, -1,  1],
       [-1, -1, -1, ..., -1,  0, -1],
       [ 1,  1,  1, ...,  1, -1,  0]])

In [22]:


## previous_weights = previous_weights + weights
